In [1]:
# Прогнозирование сердечно-сосудистых заболеваний
## Автор: Диас К.
## Дата: 18 ноября 2025

In [ ]:
# Установка SHAP (один раз)
!pip install shap -q
# Импорт библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, f1_score, roc_auc_score, classification_report
import xgboost as xgb
import shap

%matplotlib inline
sns.set(style="whitegrid", font_scale=1.1)

# Загрузка данных
url = "https://raw.githubusercontent.com/caravanuden/cardio/master/cardio_train.csv"
df = pd.read_csv(url, sep=';')

from google.colab import files
uploaded = files.upload()
df = pd.read_csv('cardio_train.csv', sep=';')

df = df.drop('id', axis=1)  # id не нужен
df.head()

print("Размер датасета:", df.shape)
print("\nПропуски:\n", df.isnull().sum().sum())  # 0 пропусков
print("\nБаланс классов:")
print(df['cardio'].value_counts(normalize=True))

# Преобразование возраста в годы
df['age_years'] = (df['age'] // 365)
df = df.drop('age', axis=1)

# Расчет BMI
df['bmi'] = df['weight'] / (df['height'] / 100)**2

# Распределение целевой переменной
plt.figure(figsize=(6,4))
sns.countplot(x='cardio', data=df)
plt.title('Распределение наличия сердечно-сосудистых заболеваний')
plt.xlabel('0 — нет, 1 — есть')
plt.show()

# Гистограммы числовых признаков
df.hist(figsize=(12,10))
plt.suptitle('Распределения признаков')
plt.tight_layout()
plt.show()

# Корреляционная матрица
plt.figure(figsize=(10,8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Корреляционная матрица признаков')
plt.show()

# Влияние категориальных факторов
fig, axes = plt.subplots(2, 2, figsize=(12,10))
sns.countplot(x='cholesterol', hue='cardio', data=df, ax=axes[0,0])
axes[0,0].set_title('Холестерин и заболевание')
sns.countplot(x='gluc', hue='cardio', data=df, ax=axes[0,1])
axes[0,1].set_title('Глюкоза и заболевание')
sns.countplot(x='smoke', hue='cardio', data=df, ax=axes[1,0])
axes[1,0].set_title('Курение и заболевание')
sns.countplot(x='active', hue='cardio', data=df, ax=axes[1,1])
axes[1,1].set_title('Физическая активность и заболевание')
plt.tight_layout()
plt.show()

# Кодирование пола (1 → женский, 2 → мужской → 0/1)
df['gender'] = df['gender'] - 1

# Признаки и целевая переменная
features = ['age_years', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo',
            'cholesterol', 'gluc', 'smoke', 'alco', 'active', 'bmi']
X = df[features]
y = df['cardio']

# Разделение
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42, stratify=y)

# Масштабирование (для логистической регрессии)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Функция для вывода метрик
def evaluate_model(model, X_test_data, name):
    if 'scaled' in name:
        pred = model.predict(X_test_scaled)
        prob = model.predict_proba(X_test_scaled)[:,1]
    else:
        pred = model.predict(X_test_data)
        prob = model.predict_proba(X_test_data)[:,1]

    print(f"\n=== {name} ===")
    print(f"Accuracy: {accuracy_score(y_test, pred):.4f}")
    print(f"Recall:   {recall_score(y_test, pred):.4f}")
    print(f"F1-score: {f1_score(y_test, pred):.4f}")
    print(f"ROC-AUC:  {roc_auc_score(y_test, prob):.4f}")

    return [accuracy_score(y_test, pred), recall_score(y_test, pred),
            f1_score(y_test, pred), roc_auc_score(y_test, prob)]

            # 1. Логистическая регрессия
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train_scaled, y_train)
metrics_logreg = evaluate_model(logreg, None, "Logistic Regression (scaled)")

# 2. Random Forest
rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)
metrics_rf = evaluate_model(rf, X_test, "Random Forest")

# 3. XGBoost
xgb_model = xgb.XGBClassifier(n_estimators=200, random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)
metrics_xgb = evaluate_model(xgb_model, X_test, "XGBoost")

results = pd.DataFrame({
    'Модель': ['Logistic Regression', 'Random Forest', 'XGBoost'],
    'Accuracy': [metrics_logreg[0], metrics_rf[0], metrics_xgb[0]],
    'Recall': [metrics_logreg[1], metrics_rf[1], metrics_xgb[1]],
    'F1-score': [metrics_logreg[2], metrics_rf[2], metrics_xgb[2]],
    'ROC-AUC': [metrics_logreg[3], metrics_rf[3], metrics_xgb[3]]
})
results.sort_values('ROC-AUC', ascending=False)

# Feature importance для Random Forest
importances = pd.Series(rf.feature_importances_, index=features).sort_values()
plt.figure(figsize=(8,6))
importances.plot.barh()
plt.title('Важность признаков (Random Forest)')
plt.xlabel('Важность')
plt.show()

# Feature importance для XGBoost
xgb.plot_importance(xgb_model)
plt.title('Важность признаков (XGBoost)')
plt.show()

# SHAP для XGBoost
explainer = shap.Explainer(xgb_model)
shap_values = explainer(X_test)
shap.summary_plot(shap_values, X_test, feature_names=features, plot_type="bar")
shap.summary_plot(shap_values, X_test, feature_names=features)
